<a href="https://colab.research.google.com/github/AslanDevbrat/NLP/blob/main/Neural_Machine_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## DATASET

In [ ]:
!wget http://manythings.org/anki/fra-eng.zip
!unzip /content/fra-eng.zip

--2022-07-20 11:01:03--  http://manythings.org/anki/fra-eng.zip
Resolving manythings.org (manythings.org)... 173.254.30.110
Connecting to manythings.org (manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://www.manythings.org/anki/fra-eng.zip [following]
--2022-07-20 11:01:03--  http://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Reusing existing connection to manythings.org:80.
HTTP request sent, awaiting response... 200 OK
Length: 6612164 (6.3M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   6.31M  20.2MB/s    in 0.3s    

2022-07-20 11:01:03 (20.2 MB/s) - ‘fra-eng.zip’ saved [6612164/6612164]



Archive:  /content/fra-eng.zip
  inflating: _about.txt              
  inflating: fra.txt                 


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.preprocessing import text_dataset_from_directory
from tensorflow.keras.layers import SimpleRNN,LSTM, Dense, Reshape, Embedding,GRU,Bidirectional, Input
import numpy as np
import re
import string
import nltk
import numpy as np
from matplotlib import pyplot as plt
import pandas
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [191]:
path= "/content/fra.txt"
NUM_EXAMPLES = 250
text_dataset = tf.data.TextLineDataset(path).take(NUM_EXAMPLES)


In [192]:
def selector(input_text):
  return tf.strings.split(input_text, '\t')[0:1], 'starttoken ' + tf.strings.split(input_text, '\t')[1:2], tf.strings.split(input_text, '\t')[1:2]

text_dataset = text_dataset.map(selector)


In [193]:


for i in text_dataset.take(1):
  print(i)

(<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Go.'], dtype=object)>, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'starttoken Va !'], dtype=object)>, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Va !'], dtype=object)>)


In [194]:
def preprocess_sentences(input_data):
    '''
    Task: Preprocess sentences or standardize the sentences
    Input: raw reviews
    output: standardized reviews
    '''
    output=tf.strings.lower(input_data)
    outputs=tf.strings.regex_replace(output,"<[^>]+>","")
    outputs=tf.strings.regex_replace(output,"<[%s]"%re.escape(string.punctuation)," ")
    outputs=tf.strings.regex_replace(output,"  "," ")
    
    return output
SEQUENCE_LENGTH = 10
vectorize_input_layer = TextVectorization(
      standardize = preprocess_sentences,
      output_sequence_length = SEQUENCE_LENGTH,

      )
      
vectorize_output_layer = TextVectorization(
    standardize = preprocess_sentences,
    output_sequence_length = SEQUENCE_LENGTH,
)

vectorize_pre_output_layer = TextVectorization(
    standardize = preprocess_sentences,
    output_sequence_length = SEQUENCE_LENGTH,
)

In [195]:
training_data = text_dataset.map(lambda x,y, z :x)
vectorize_input_layer.adapt(training_data)

training_data = text_dataset.map(lambda x, y, z :y)
vectorize_pre_output_layer.adapt(training_data)

training_data = text_dataset.map(lambda x, y, z :z)
vectorize_output_layer.adapt(training_data)


In [196]:
VOCAB_INPUT_SIZE  = len(vectorize_input_layer.get_vocabulary())
VOCAB_OUTPUT_SIZE = len(vectorize_output_layer.get_vocabulary())
VOCAB_PRE_OUTPUT_SIZE=len(vectorize_pre_output_layer.get_vocabulary())
print(VOCAB_OUTPUT_SIZE)

270


In [197]:
vectorize_pre_output_layer.get_vocabulary()

['',
 '[UNK]',
 'starttoken',
 '!',
 'je',
 'soyez',
 'sois',
 "j'ai",
 'le',
 '?',
 'vos',
 'de',
 'À',
 'tomber',
 'suis',
 'nous',
 'camp',
 "c'est",
 'à',
 'Ça',
 'va',
 'question',
 'pas',
 'merci',
 'maintenant.',
 'hors',
 'fiche',
 'en',
 'dégage\u202f!',
 'du',
 'casse-toi.',
 'équitable',
 'vraiment',
 'viens\xa0!',
 'venez\u202f!',
 'tom.',
 'te',
 'sortez\u202f!',
 'sors.',
 'saute.',
 'santé',
 'salut',
 'prenez',
 'pars',
 'oust',
 'on',
 'là.',
 'j’ai',
 'juste',
 'jambes',
 'gentille',
 'gentil',
 'gagné.',
 'fuyons',
 'fuyez',
 'filez',
 'file',
 'essaye.',
 'décampe',
 'disparais\u202f!',
 'disparais',
 'dans',
 "d'ici.",
 'cous',
 'cours\u202f!',
 'cours',
 'courez\u202f!',
 'camp.',
 'calme',
 'bras',
 'bouge',
 'attends',
 'attendez.',
 'attendez',
 'ah',
 'équitables',
 'Écoutez',
 'y',
 'waouh\xa0!',
 'wah\xa0!',
 'vraiment\u202f?',
 'vrai,',
 'vrai',
 'votre',
 'vers',
 'va\xa0!',
 'vas-y',
 'vais',
 'va.',
 'va-t’en\xa0!',
 'va,',
 'tricote.',
 'tranquille.',
 

In [198]:


def vectorizer(x, y, z):
  return {'in1': tf.squeeze(vectorize_input_layer(x), 0) , 'in2': tf.squeeze(vectorize_pre_output_layer(y),0) }, tf.squeeze(tf.one_hot(vectorize_output_layer(z), depth = VOCAB_OUTPUT_SIZE), 0)

dataset = text_dataset.map(vectorizer)

In [199]:
for i in dataset.take(1):

  print(i)

({'in1': <tf.Tensor: shape=(10,), dtype=int64, numpy=array([46,  0,  0,  0,  0,  0,  0,  0,  0,  0])>, 'in2': <tf.Tensor: shape=(10,), dtype=int64, numpy=array([ 2, 20,  3,  0,  0,  0,  0,  0,  0,  0])>}, <tf.Tensor: shape=(10, 270), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)>)


In [200]:
VALIDATION_RATIO = 0.8
VALIDATION_BRIDGE = int(VALIDATION_RATIO* NUM_EXAMPLES)
dataset = dataset.shuffle(NUM_EXAMPLES)
train_dataset = dataset.take(VALIDATION_BRIDGE)
validation_dataset = dataset.skip(VALIDATION_BRIDGE)

In [201]:
BATCH_SIZE = 64
train_dataset=train_dataset.batch(BATCH_SIZE).cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset=validation_dataset.batch(BATCH_SIZE).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

# MODELLING

In [202]:
SEQUENCE_LENGTH = 10
EMBEDDING_DIM = 16
enc_size = 300
dec_size = 100


enc_input = Input((SEQUENCE_LENGTH,),name = 'in1')
dec_input = Input((SEQUENCE_LENGTH,), name = 'in2')

x = Embedding(VOCAB_INPUT_SIZE, EMBEDDING_DIM)(enc_input)

encoder = LSTM(enc_size, return_sequences= False, return_state = True)
_ , h, c = encoder(x)

x = Embedding(VOCAB_PRE_OUTPUT_SIZE, EMBEDDING_DIM)(dec_input)

decoder = LSTM(dec_size, return_sequences=True, return_state = True)
h = Dense(dec_size)(h)
c = Dense(dec_size)(c)

x, h, c = decoder(x, [h, c])
x = Dense(VOCAB_OUTPUT_SIZE, activation = "softmax")(x)

model = tf.keras.Model([enc_input, dec_input], x)
model.summary()





Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 in1 (InputLayer)               [(None, 10)]         0           []                               
                                                                                                  
 embedding_14 (Embedding)       (None, 10, 16)       1392        ['in1[0][0]']                    
                                                                                                  
 in2 (InputLayer)               [(None, 10)]         0           []                               
                                                                                                  
 lstm_14 (LSTM)                 [(None, 300),        380400      ['embedding_14[0][0]']           
                                 (None, 300),                                               

In [206]:
class Bleu(tf.keras.metrics.Metric):

  def __init__(self, name = 'bleu_score'):
    super(Bleu, self).__init__()
    self.add = 0
    self.total = 0
  
  def update_state(self, y_true, y_pred, sample_weight = None):
    #
    y_true = tf.argmax(y_true, -1)
    y_pred = tf.argmax(y_pred, -1)

    for i, j in zip(y_pred, y_true):
      tf.autograph.experimental.set_loop_options()
      self.total += tf.math.count_nonzero(i)

      for word in i:
        if (word == 0):
          break
        for q in range(len(j)):
          if word == j[q]:
            self.add += 1
            j = tf.boolean_mask(j, [False if  y==q else True for y in range(len(j)) ])
            break
          
  def result(self):
    return self.add/self.total
  
  def penalty(self):
    if self.add > self.total:
      return 1
    else:
      return tf.math.exp(1- (self.total/self.add))

In [207]:
LR = 1e-3
EPOCH = 100

model.compile(
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits = True),
    optimizer = tf.keras.optimizers.Adam(lr = LR,),
    metrics = [Bleu()],
    run_eagerly = True,
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [208]:
checkpoint_filepath  = "seq2seq.hdf5"

#checkpoint_filepath="imdb/basic_rnn.hdf5"
callback=tf.keras.callbacks.ModelCheckpoint(
   filepath=checkpoint_filepath,### WHERE BEST WEIGHTS ARE STORED
    save_weights_only=True,
   monitor="loss",
   mode='min',
    save_best_only=True)
#Train model on dataset
history=model.fit(train_dataset, validation_data = val_dataset, verbose=1,epochs=EPOCH,callbacks=[callback])#

Epoch 1/100


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: not enough values to unpack (expected 2, got 0)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: not enough values to unpack (expected 2, got 0)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: not enough values to unpack (expected 2, got 0)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
4/4 [==============================] - 7s 624ms/step - loss: 1.1217 - bleu_21: 0.2850 - val_loss: 1.1382 - val_bleu_21: 0.3000
Epoch 2/100
4/4 [==============================] - 1s 360ms/step - loss: 1.0992 - bleu_21: 0.3492 - val_loss: 1.1275 - val_bleu_21: 0.3492
Epoch 3/100
4/4 [==============================] - 1s 287ms/step - loss: 1.0851 - bleu_21: 0.3492 - val_loss: 1.1230 - val_bleu_21: 0.3492
Epoch 4/100
4/4 [==============================] - 1s 289ms/step - loss: 1.0765 - bleu_21: 0.3492 - val_loss: 1.1202 - val_bleu_21: 0.3492
Epoch 5/100
4/4 [==============================] - 1s 295ms/step - loss: 1.0699 - bleu_21: 0.3492 - val_loss: 1.1172 - val_bleu_21: 0.3492
Epoch 6/

KeyboardInterrupt: ignored

In [ ]:
 vectorize_output_layer.get_vocabulary()

## Testing

In [209]:
test_data = tf.data.Dataset.from_tensor_slices([['i will try']])
init_test_data = tf.data.Dataset.from_tensor_slices([["starttoken je soyez"]])

input_test_data = test_data.map(vectorize_input_layer)
pre_output_test_data = init_test_data.map(vectorize_pre_output_layer)
for i in input_test_data.take(1):
  print(i)
  in_1 = i

for i in pre_output_test_data.take(1):
  print(i)
  in_2 =i

tf.Tensor([[ 4  1 38  0  0  0  0  0  0  0]], shape=(1, 10), dtype=int64)
tf.Tensor([[2 4 5 0 0 0 0 0 0 0]], shape=(1, 10), dtype=int64)


In [210]:
def get_output(in_1, in_2):
  return tf.argmax(model.predict([in_1, in_2]), -1)
output = get_output(in_1, in_2)
output

<tf.Tensor: shape=(1, 10), dtype=int64, numpy=array([[4, 0, 0, 0, 0, 0, 0, 0, 0, 0]])>

In [155]:
for i in range(SEQUENCE_LENGTH):
  print(vectorize_output_layer.get_vocabulary()[output[0][i]])

soyez











In [151]:
vectorize_pre_output_layer.get_vocabulary()[output[0][0]]

'je'

In [ ]:
vectorize_pre_output_layer.get_vocabulary()

['',
 '[UNK]',
 'starttoken',
 '!',
 'je',
 'suis',
 "c'est",
 '?',
 "j'ai",
 'tom',
 'est',
 'à',
 'de',
 'le',
 'me',
 'il',
 'tom.',
 'nous',
 'en',
 'soyez',
 'la',
 'ne',
 'a',
 'vous',
 'restez',
 'reste',
 'pas',
 'un',
 'Ça',
 'qui',
 'ça.',
 'sois',
 'pas.',
 'va',
 'est-ce',
 'du',
 'les',
 'au',
 'À',
 'j’ai',
 'fais',
 'ça',
 "l'ai",
 "j'en",
 'te',
 'ici.',
 'elle',
 'calme.',
 'ai',
 'prends',
 'bien.',
 'bien',
 'plus',
 'fait',
 'tu',
 'prenez',
 'faites',
 'suis-je',
 'on',
 'comme',
 'bouge',
 'que',
 'perdu.',
 'laissez-moi',
 'avons',
 'venez',
 'tranquille.',
 'malade.',
 "m'en",
 'laisse',
 'attention.',
 'allez',
 'été',
 'vais',
 'une',
 'là.',
 'ils',
 'elles',
 'ceci.',
 'aller',
 'moi',
 "j'étais",
 'y',
 'vu.',
 'vos',
 'viens',
 'toi.',
 'sens',
 'regarde',
 'laisse-moi',
 'demande',
 'chez',
 'bon',
 'vraiment',
 'vers',
 'trouve',
 'triste.',
 'tranquille',
 'train',
 'sommes',
 'soin',
 'sans',
 'peux',
 'pars',
 'ont',
 'gagné.',
 'décampe',
 'comment',